In [1]:
import tweepy
import pandas as pd

In [3]:
from app_cred import CONSUMER_KEY, CONSUMER_SECRET
from app_cred import ACCESS_TOKEN, ACCESS_TOKEN_SECRET

In [4]:
# auth stuff
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [58]:
def limit_handled(cursor):
    while True:
        try:
            yield next(cursor)
        except tweepy.RateLimitError:
            time.sleep(15 * 60)
        except StopIteration:
            break

In [71]:
def get_all_tweets(handle, n):
    '''this function is supposed to return all possible tweets from a user in a df
    Handle is the handle of the account and n is the number of wished tweets'''
    timeline = tweepy.Cursor(api.user_timeline, screen_name=handle, tweet_mode="extended")
    tweet_list = [status._json for status in limit_handled(timeline.items(n))]
    return pd.DataFrame(tweet_list)

In [73]:
df = get_all_tweets('greenpeacedk', 5)

In [27]:
handle = 'greenpeacedk'
# this gets 200 most recent tweets: this is the amount allowed through this method
tweets = api.user_timeline(screen_name=handle, count=200, tweet_mode='extended')